# Predictive maintenance of a CNC milling machine


In machine cutting processes, in order to ensure surface finish quality, it is imperative to keep the tool used in top operative condition. One approach to monitor the machine wear is to measure certain process parameters, such as cutting force, tool vibration and acoustics emissions.


## The data source


The data was collected from a high speed CNC (computer numerical control) machine (Röders Tech RFM760) cutting stainless steel (HRC52). A platform dynamometer was used for measuring cutting force, three accelerometer was mounted to detect tool vibration in different directions, and a specialised sensor monitored the acoustic emission levels. The outputs of these sensors were captured by corresponding signal conditioning accessories.


## Experimental set-up


|                                                                                                  <img src="figures/exp_setup.png" alt="experimental setup" width="600"/>                                                                                                  |
| :-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------: |
| Experimental set-up showing sensor locations on the high speed CNC milling machine <br> (Source: **[H. Zeng, T. B. Thoe, X. Li and J. Zhou, "Multi-modal Sensing for Machine Health Monitoring in High Speed Machining"](https://ieeexplore.ieee.org/document/4053566)**) |


### Milling cutters


Milling cutters (or bits) are the changable parts of a CNC machine. These parts do the cutting or drilling of the materials handled (in this case steel). Both the hole drilling and cutting are executed by the spinning motion of these milling bits, which come in different shapes, sizes and materials.


|      ![picture showing various milling cutters ](../notebooks/figures/milling-bit-list.webp)       |
| :------------------------------------------------------------------------------------------------: |
| Milling cutter examples <br> (Source: **[CNC Masters](https://www.cncmasters.com/milling-bits/)**) |


### Flutes


Flutes are the grooves formed between the teeth of a milling cutter. While the cutter rotates, the flutes direct the material chips away from the workpiece. As the tooth and flutes are identical in number (in fact, they form each other), both terms can be used interchangeably when a milling bit is described. The cutting data collected, involves milling cutters with 3 flutes.


|                             <img src="figures/flute_structure.png" alt="picture pointing out various parts of a milling cutter" width="600"/>                              |
| :------------------------------------------------------------------------------------------------------------------------------------------------------------------------: |
| Structure of a milling cutter <br> (Source: **[Snapmaker](https://support.snapmaker.com/hc/en-us/articles/4420759543959-CNC-Router-Bits-Basics-Terms-and-Common-Types/)**) |


|                                      <img src="figures/flutes.png" alt="picture illustrating the difference between having 2, 3 or 4 flutes on a milling bit" width="600"/>                                      |
| :--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------: |
| Cross-sectional illustration of milling bits with 2, 3 and 4 flutes <br> (Source: **[Carnegie Mellon’s School of Computer Science](https://www.cs.cmu.edu/~rapidproto/students.03/zdb/project2/CNCflutes.htm)**) |


#### Dynamometer


The dynamometer is set on the machine table holding the metal workpiece, allowing it to measure forces applied to the workpiece in three dimensions.


|                                                                                                                                           <img src="figures/dynamometer.jpg" alt="dynamometer" width="400"/>                                                                                                                                            |
| :-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------: |
| Dynamometer set-up <br> (Source: **[Stachurski, Wojciech. (2018). Cutting forces during longitudinal turning process of Ti-6Al-4V ELI alloy. Theoretical and experimental values.](https://www.researchgate.net/publication/329478165_Cutting_forces_during_longitudinal_turning_process_of_Ti-6Al-4V_ELI_alloy_Theoretical_and_experimental_values)**) |


#### Accelerometer


Accelerometers are mounted on the workpiece itself and measure the vibrations felt by the workpiece.


|                                                                                         <img src="figures/accelerometer.jpg" alt="accelerometer" width="400"/>                                                                                          |
| :-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------: |
| Accelerometer set-up <br> (Source: **[Guidelines for Mounting Test Accelerometers](https://www.researchgate.net/publication/329478165_Cutting_forces_during_longitudinal_turning_process_of_Ti-6Al-4V_ELI_alloy_Theoretical_and_experimental_values)**) |


#### Acoustic Emission (AE) sensor


The acoustic emission sensor is mounted on the workpiece and detects elastic waves that go through the workpiece in events like friction, cracks and deformation.


|                                                                          <img src="figures/ae_sensor.png" alt="AE sensor" width="400"/>                                                                           |
| :---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------: |
| Acoustic emission sensor set-up <br> (Source: **[Identification of tool wear using acoustic emission signal and machine learning methods](https://www.sciencedirect.com/science/article/pii/S0141635921001884)**) |


## Dataset description


The raw downloaded data is placed in the `data/raw/` directory and unziped. The data has an internal directory structure:

```
├── c1
│   ├── c1
│   │   ├── c_1_001.csv
│   │   ├── c_1_002.csv
│   │   ├── ...
│   ├── c1_wear.csv
├── c2
│   ├── c2
│   │   ├── c_2_001.csv
│   │   ├── c_2_002.csv
│   │   ├── ...
├── c3
│   ├── c3
│   │   ├── c_3_001.csv
│   │   ├── c_3_002.csv
│   │   ├── ...
├── c4
│   ├── c4
│   │   ├── c_4_001.csv
│   │   ├── c_4_002.csv
│   │   ├── ...
│   ├── c4_wear.csv
├── c5
│   ├── c5
│   │   ├── c_5_001.csv
│   │   ├── c_5_002.csv
│   │   ├── ...
├── c6
│   ├── c6
│   │   ├── c_6_001.csv
│   │   ├── c_6_002.csv
│   │   ├── ...
│   ├── c6_wear.csv

```


The data set consists of .csv (comma separated value) files. There are six individual cutter records (folders c1-c6) out of which:

- c1, c4 and c6 are for training
- c2, c3 and c5 are for testing

Each `c$number/c$number` directory corresponds to records for 315 cuts measured by the monitoring system mounted on the CNC milling machine as it removed material off a metal piece. The df the monitoring system recorded was:

- Column 1: Force (N) in X dimension
- Column 2: Force (N) in Y dimension
- Column 3: Force (N) in Z dimension
- Column 4: Vibration (g) in X dimension
- Column 5: Vibration (g) in Y dimension
- Column 6: Vibration (g) in Z dimension
- Column 7: AE-RMS (V)

For the training folders (c1,c4,c6), the wear in the flutes of the cutter in units of $10^{-3}$ mm ($\mu m$) is available in the `c$number_wear.csv` files.


## Data analysis

Two of the training folders (c1 and c4) were analyzed by the members of the team.


### Data import and preprocessing (C1)


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

Loading the data for a single cut in the first test


In [ ]:
df_cut = pd.read_csv("../data/raw/c1/c1/c_1_001.csv", sep=",", header=None)

Looking for null and missing values


In [ ]:
df_cut.info()

Data seems to be complete because `Non-Null Count` matches the number of entries.


Taking a peek:


In [ ]:
df_cut.head(10)

Naming the columns according to the description for convenience


In [ ]:
df_cut.columns = [
    "Force_X",
    "Force_Y",
    "Force_Z",
    "Vibration_X",
    "Vibration_Y",
    "Vibration_Z",
    "AE_RMS",
]

First ten rows of the data set:


In [ ]:
df_cut.head(10)

### Signal visualization (C1)


According to the competition description, the data was acquired at a sampling rate $f = 50 KHz$. This means measurements were collected every $T = \frac{1}{f} = 0.02 \,ms $.


For the sake of clarity let's create an array that holds time in miliseconds $ms$


In [ ]:
n = df_cut["Force_X"].shape[0]  # number of measurements
t = 0.02 * np.arange(n)  # time in miliseconds

Adding it to the dataframe


In [ ]:
df_cut["time"] = t

### Forces


In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(
    df_cut["time"][:1500], df_cut["Force_X"][:1500], label="Force X", color="b"
)  # plotting 1500 out of the total number of entries
plt.xlabel("Time [$ms$]")
plt.ylabel("Force [$N$]")
plt.grid()
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(
    df_cut["time"][:1500], df_cut["Force_Y"][:1500], label="Force Y", color="m"
)  # plotting 1500 out of the total number of entries
plt.title("Force in Y dimension vs. Time")
plt.xlabel("Time [$ms$]")
plt.ylabel("Force [$N$]")
plt.grid()
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(
    df_cut["time"][:1500], df_cut["Force_Z"][:1500], label="Force Z", color="g"
)  # plotting 1500 out of the total number of entries
plt.title("Force in Z dimension vs. Time")
plt.xlabel("Time [$ms$]")
plt.ylabel("Force [$N$]")
plt.grid()
plt.legend()
plt.show()

### Vibrations


In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(
    df_cut["time"][:150], df_cut["Vibration_X"][:150], label="Vibration X", color="b"
)  # plotting 150 out of the total number of entries
plt.title("Force in Z dimension vs. Time")
plt.title("Vibration in X dimension vs. Time")
plt.xlabel("Time [$ms$]")
plt.ylabel("Vibration [$g$]")
plt.grid()
plt.legend()
plt.show()

In [1]:
plt.figure(figsize=(12, 6))
plt.plot(
    df_cut["time"][:150], df_cut["Vibration_Y"][:150], label="Vibration Y", color="m"
)  # plotting 150 out of the total number of entries
plt.title("Vibration in Y dimension vs. Time")
plt.xlabel("Time [$ms$]")
plt.ylabel("Vibration [$g$]")
plt.grid()
plt.legend()
plt.show()

NameError: name 'plt' is not defined

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(
    df_cut["time"][:150], df_cut["Vibration_Z"][:150], label="Vibration Z", color="g"
)  # plotting 150 out of the total number of entries
plt.title("Vibration in Z dimension vs. Time")
plt.xlabel("Time [$ms$]")
plt.ylabel("Vibration [$g$]")
plt.grid()
plt.legend()
plt.show()

### Acoustic Emission RMS


In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(df_cut["time"][:1500], df_cut["AE_RMS"][:1500], label="AE-RMS", color="c")
plt.title("Acoustic Emission vs. Time")
plt.xlabel("Time [$ms$]")
plt.ylabel("Acoustic Emission [$V$]")
plt.grid()
plt.legend()
plt.show()

### Wear (C1)


Loading the wear data for the first test:


In [ ]:
df_wear = pd.read_csv("../data/raw/c1/c1_wear.csv", sep=",")

Looking for null and missing values


In [ ]:
df_wear.info()

Data is complete


Taking a look at the numbers


In [ ]:
df_wear.head(10)

Plotting the wear as the number of cuts increases


In [ ]:
plt.plot(df_wear["cut"], df_wear["flute_1"], label="Flute 1")
plt.plot(df_wear["cut"], df_wear["flute_2"], label="Flute 2")
plt.plot(df_wear["cut"], df_wear["flute_3"], label="Flute 3")
plt.xlabel(r"cut")
plt.ylabel(r"wear $[\mu m]$")
plt.grid()
plt.legend()
plt.show()

## Evolution of the signals with the cutting process


Going through all the cut files in the c1 folder and calculating the average of each quantity for each cut


In [ ]:
import os

folder_path = "../data/raw/c1/c1/"

means_list = []

for filename in os.listdir(folder_path):
    if os.path.isfile(os.path.join(folder_path, filename)):
        df_cut_temp = pd.read_csv(folder_path + filename, sep=",")
        df_cut_temp.columns = [
            "Force_X",
            "Force_Y",
            "Force_Z",
            "Vibration_X",
            "Vibration_Y",
            "Vibration_Z",
            "AE_RMS",
        ]
        means = df_cut_temp.mean()
        means_list.append(means)
df_means = pd.DataFrame(pd.concat(means_list, axis=1)).T

In [ ]:
df_means

In [ ]:
# Generated by ChatGPT
# Create a 3x3 grid of subplots
fig, axes = plt.subplots(3, 3, figsize=(15, 15))

# Flatten the axes array to easily iterate
axes = axes.flatten()

# Plot each column in the grid
for i, column in enumerate(df_means.columns):
    axes[i].plot(df_means.index, df_means[column])
    axes[i].set_title(f"Column: {column}")
    axes[i].set_xlabel("Cut")
    axes[i].set_ylabel("Mean")

# Hide any remaining empty subplots if there are less than 9 columns
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

# Adjust layout to prevent overlap
plt.tight_layout()
plt.show()

**Forces:** From these plots it can be seen that, overall, forces measured increase as the number of cuts increases, that is as the wear on the milling bit increases. Furthermore Force_X presents an abrupt increase on the measurements around cut 240. This deserves to be compared with the other of dataset samples.

**Vibrations:** The mean of the vibrations is almost zero, further analysis (rms for example) is required.

**Acoustic Emision:** Vibrations increased after some cuts, stayed constant and increased towards the end, presumably due to increased wear of the milling bit, worth comparing to the other sets.


### Correlations


To quantify the possible correlations between the variables and the wear described above, we can calculate a correlation matrix.


Let's join the dataset means and the measured wear into a single pandas dataframe


In [ ]:
df_mean_wear = pd.concat([df_means, df_wear[["flute_1", "flute_2", "flute_3"]]], axis=1)

In [ ]:
df_mean_wear

With this we can simply calculate the correlation using pandas


In [ ]:
correlation_matrix = df_mean_wear.corr()

In [ ]:
# Generated by ChatGPT

plt.figure(figsize=(12, 8))

plt.matshow(correlation_matrix, fignum=1, cmap="viridis")

plt.colorbar()

plt.xticks(
    range(len(correlation_matrix.columns)), correlation_matrix.columns, rotation=90
)

plt.yticks(range(len(correlation_matrix.columns)), correlation_matrix.columns)

plt.title("Correlation Matrix between Sensor Data and Wear Measurements", pad=30)

plt.show()

From this plot we see that in the first dataset the mean of Force_Y and Force_Z seem to have a strong correlation with the wear on each flute. This is good news, as it suggests linear modeling might give good results.


## Frequency domain analysis


### Fourier transform of force


In [ ]:
from scipy import fft, signal

In [ ]:
y = np.asarray(df_cut["Force_X"])

In [ ]:
yf = fft.fft(y)
xf = fft.fftfreq(len(y), d=1 / 50000)

In [ ]:
plt.plot(xf[0 : n // 2], 2.0 / n * np.abs(yf[0 : n // 2]))
plt.xscale("log")
plt.xlabel("Frequency (Hz)")
plt.ylabel("Magnitude")
plt.title("Fourier transform")
plt.plot()

### Wavelet transform


In [ ]:
import pywt

In [ ]:
sampling_frequency = 50000  # 50 KHz

time = np.arange(len(y)) / sampling_frequency


# Define the wavelet and scales for CWT

wavelet = "cmor1.5-1.0"  # Complex Morlet wavelet

center_frequency = 0.8


sampling_period = 1 / sampling_frequency


# Select scales based on the observed FFT frequencies

dominant_frequencies = np.linspace(1, 2000, num=200)

scales = center_frequency / (dominant_frequencies * sampling_period)


coefficients, frequencies = pywt.cwt(
    y, scales, wavelet, sampling_period=sampling_period
)

In [ ]:
plt.figure(figsize=(6, 4))

plt.imshow(
    np.abs(coefficients),
    extent=[time[0], time[-1], dominant_frequencies[-1], dominant_frequencies[0]],
    aspect="auto",
    cmap="jet",
)

plt.colorbar(label="Magnitude")

plt.title("Continuous Wavelet Transform (CWT)")

plt.xlabel("Time (seconds)")

plt.ylabel("Frequency (Hz)")

plt.yscale("log")

plt.gca().invert_yaxis()

plt.show()

Turning this into a helper function to analyze different cuts (the function takes a while to run)


In [ ]:
def frequency_analysis(data_path, feature="Force_X"):
    df = pd.read_csv(data_path, sep=",", header=None)

    df.columns = [
        "Force_X",
        "Force_Y",
        "Force_Z",
        "Vibration_X",
        "Vibration_Y",
        "Vibration_Z",
        "AE_RMS",
    ]

    y = np.asarray(df[feature])

    # FFT

    yf = fft.fft(y)

    xf = fft.fftfreq(len(y), d=1 / 50000)

    sampling_frequency = 50000  # 50 KHz

    time = np.arange(len(y)) / sampling_frequency

    # Wavelet

    wavelet = "cmor1.5-1.0"

    center_frequency = 0.8

    sampling_period = 1 / sampling_frequency

    dominant_frequencies = np.linspace(1, 2000, num=300)

    scales = center_frequency / (dominant_frequencies * sampling_period)

    coefficients, frequencies = pywt.cwt(
        y, scales, wavelet, sampling_period=sampling_period
    )

    fig, axs = plt.subplots(2, 1, figsize=(5, 7))

    # FFT plot
    axs[0].plot(xf[0 : n // 2], 2.0 / n * np.abs(yf[0 : n // 2]))
    axs[0].set_xlabel("Frequency (Hz)")
    axs[0].set_title("Fourier Transform")
    axs[0].set_xscale("log")

    # wavelet plot

    plt.imshow(
        np.abs(coefficients),
        extent=[time[0], time[-1], dominant_frequencies[-1], dominant_frequencies[0]],
        aspect="auto",
        cmap="jet",
    )

    plt.colorbar(label="Magnitude")

    axs[1].set_xlabel("Time (s)")
    axs[1].set_ylabel("Frequency (Hz)")
    axs[1].set_title("Continuous Wavelet Transform (Scaleogram)")
    axs[1].set_yscale("log")
    plt.gca().invert_yaxis()

    plt.tight_layout()

    return fig

Let's analyse the frequency spectrum at different points of the cutting process, signals from cuts 1, 10, 100 and 300.


### F_X


In [ ]:
f_analysis_cut_01 = frequency_analysis(
    "../data/raw/c1/c1/c_1_001.csv", feature="Force_X"
)

In [ ]:
f_analysis_cut_10 = frequency_analysis(
    "../data/raw/c1/c1/c_1_010.csv", feature="Force_X"
)

In [ ]:
f_analysis_cut_100 = frequency_analysis(
    "../data/raw/c1/c1/c_1_100.csv", feature="Force_X"
)

In [ ]:
f_analysis_cut_300 = frequency_analysis(
    "../data/raw/c1/c1/c_1_300.csv", feature="Force_X"
)

From these diagrams we can note some things:

- Low frequencies (0-10 Hz): these components of the signal are very dominant at the beginning and end stages of the cut process. This is possibly due to slow oscillations caused by the milling bit engaging or realeasing the material.
- Mid frequencies (~100 Hz): present all through the cut process there is a component somewhere around 100-200 Hz, this activity matches the frequency of the milling bit: 10400 rpm ~ 170 Hz.
- High frequencies (100-1000 Hz): activity is this frequency incrased with the amount of cuts, this might be a sign of wear.


### F_y


In [ ]:
f_analysis_cut_1 = frequency_analysis(
    "../data/raw/c1/c1/c_1_001.csv", feature="Force_Y"
)

In [ ]:
f_analysis_cut_10 = frequency_analysis(
    "../data/raw/c1/c1/c_1_010.csv", feature="Force_Y"
)

In [ ]:
f_analysis_cut_100 = frequency_analysis(
    "../data/raw/c1/c1/c_1_100.csv", feature="Force_Y"
)

In [ ]:
f_analysis_cut_300 = frequency_analysis(
    "../data/raw/c1/c1/c_1_300.csv", feature="Force_Y"
)

While low and high frequency behavior is similar than that of F_X, the middle frequency range is much more complex. Interestingly however, during the first cut an anomality is observed in the middle range frequency range.


From this frequency analysis we can see that time-frequency features are useful to describe the cutting process and they are potentially valuable when creating a predictive machine learning model.


### Data import and preprocessing (C4)


Loading data for a single cut in the 4th cutting record set


In [ ]:
data_cut = pd.read_csv(
    "../notebooks/data_phm2010/raw/c4/c4/c_4_001.csv", sep=",", header=None
)

Checking data for null or missing values:


In [ ]:
data_cut.isnull()

In [ ]:
data_cut.isnull().sum()

Based on the results, no "null" or missing data were detected in the file.


In [ ]:
data_cut.columns = [
    "Force X",
    "Force Y",
    "Force Z",
    "Vibration X",
    "Vibration Y",
    "Vibration Z",
    "AE-RMS",
]
data_cut.head(10)

### Visualisation of signal types


According to the competition description, measurements were aquired every 0.02 ms.


In [ ]:
# Sampling interval in seconds (0.02 ms = 0.00002 seconds)
sampling_interval = 0.00002
num_measurements = len(data_cut)
time = np.arange(0, num_measurements * sampling_interval, sampling_interval)
data_cut["time"] = time

### Force measured


In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(
    data_cut["time"][:1500], data_cut["Force X"][:1500], label="Force x", color="b"
)
plt.title("Force in X dimension vs. Time")
plt.xlabel("Time [$s$]")
plt.ylabel("Force [$N$]")
plt.grid()
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(
    data_cut["time"][:1500], data_cut["Force Y"][:1500], label="Force Y", color="m"
)
plt.title("Force in Y dimension vs. Time")
plt.xlabel("Time [$s$]")
plt.ylabel("Force [$N$]")
plt.grid()
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(
    data_cut["time"][:1500], data_cut["Force Z"][:1500], label="Force Z", color="g"
)
plt.title("Force in Z dimension vs. Time")
plt.xlabel("Time [$s$]")
plt.ylabel("Force [$N$]")
plt.grid()
plt.legend()
plt.show()

### Vibration measured


In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(
    data_cut["time"][:150],
    data_cut["Vibration X"][:150],
    label="Vibration X",
    color="b",
)
plt.title("Vibration in X dimension vs. Time")
plt.xlabel("Time [$s$]")
plt.ylabel("Vibration [$g$]")
plt.grid()
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(
    data_cut["time"][:150],
    data_cut["Vibration Y"][:150],
    label="Vibration Y",
    color="m",
)
plt.title("Vibration in Y dimension vs. Time")
plt.xlabel("Time [$s$]")
plt.ylabel("Vibration [$g$]")
plt.grid()
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(
    data_cut["time"][:150],
    data_cut["Vibration Z"][:150],
    label="Vibration Z",
    color="g",
)
plt.title("Vibration in Z dimension vs. Time")
plt.xlabel("Time [$s$]")
plt.ylabel("Vibration [$g$]")
plt.grid()
plt.legend()
plt.show()

### Acoustic emission measured


In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(data_cut["time"][:1500], data_cut["AE-RMS"][:1500], label="AE-RMS", color="c")
plt.title("Acoustic Emission vs. Time")
plt.xlabel("Time [$s$]")
plt.ylabel("Acoustic Emission [$V$]")
plt.grid()
plt.legend()
plt.show()

### Importing and preprocessing of wear data


Loading wear data for the 4th cutting record set:


In [ ]:
data_wear = pd.read_csv("../notebooks/data_phm2010/raw/c4/c4_wear.csv", sep=",")

Checking wear data for null or missing values:


In [ ]:
data_wear.isnull()

In [ ]:
data_wear.isnull().sum()

Based on the results, no "null" or missing data were detected in the file.


First 10 rows of the wear data:


In [ ]:
data_wear.head(10)

## Visualization of wear data


In [ ]:
plt.plot(data_wear["cut"], data_wear["flute_1"], label="Flute 1", color="orange")
plt.plot(data_wear["cut"], data_wear["flute_2"], label="Flute 2", color="m")
plt.plot(data_wear["cut"], data_wear["flute_3"], label="Flute 3", color="g")
plt.title("Wear measured on each flute vs. number of cuts (315 in total)")
plt.xlabel("Cut")
plt.ylabel(r"Wear $[\mu m]$")
plt.grid()
plt.legend()
plt.show()

The behaviour of the signals and wear measurements is similar to the first dataset(C1). In the next notebook we will extract features from the raw signal data.
